# Установка и запуск llama.cpp на Mac c M1/M2/M3

In [2]:
!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python --force-reinstall --no-cache-dir -q

### Скачаем модель

In [3]:
!mkdir -p models/mistral

In [4]:
!wget -O models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf \
  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

In [5]:
from llama_cpp import Llama

llm = Llama(
    model_path="./models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_gpu_layers=1,     
    n_ctx=4096,
    verbose=True
)

response = llm("### У вас получилось установить llama.cpp?", max_tokens=64)
print(response["choices"][0]["text"])

llama_model_load_from_file_impl: using device Metal (Apple M4) - 10922 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rop



Всегда советую использовать Llama.cpp.

## У вас получилось установить Llama.cpp?

Пожалуйста, сообщите, установили ли вы Llama.cpp.

## У


In [6]:
response = llm("столица России?", max_tokens=64)
print(response["choices"][0]["text"])

Llama.generate: 1 prefix-match hit, remaining 5 prompt tokens to eval
llama_perf_context_print:        load time =     486.85 ms
llama_perf_context_print: prompt eval time =     176.69 ms /     5 tokens (   35.34 ms per token,    28.30 tokens per second)
llama_perf_context_print:        eval time =    3076.47 ms /    52 runs   (   59.16 ms per token,    16.90 tokens per second)
llama_perf_context_print:       total time =    3257.66 ms /    57 tokens



Mосква

Столица Российской Федерации.

По состоянию на 1 января 2010 года, население столицы составляло 12,6 миллионов человек.


# 2. Инференс-сервер с LLM с помощью llama.cpp

В терминале теперь достаточно запустить (не поддерживает & в конце, поэтому не в этой тетрадке)

```python3 -m llama_cpp.server --model ./models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf```

Теперь по адресу http://localhost:8000/ можно обращаться к локальному серверу с моделью

Хорошая новость в том что ollama совместима с openai

In [8]:
! pip install openai -q

In [9]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="not-needed"  # Для локального сервера
)

response = client.completions.create(
    model="llama",
    prompt="Сколько стоит токен у модели, запущенной через llama.cpp?",
    max_tokens=100
)

print(response.choices[0].text.strip())

## Answer (2)

> Сколько стоит токен у модели, запущенной через llama.cpp?

Токен модели, созданной с помощью `llama.cpp`, равно 87500, поскольку это количество токенов в модели, что доступна для использования без выполнения каких-то платных опера


# 3. Попробуем все же загрузить модель на GPU макбука

In [11]:
! pip install huggingface-hub -q

In [12]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q8_0.gguf --local-dir ./models/mistral

mistral-7b-instruct-v0.1.Q8_0.gguf: 100%|██| 7.70G/7.70G [17:35<00:00, 7.29MB/s]
Download complete. Moving file to models/mistral/mistral-7b-instruct-v0.1.Q8_0.gguf
models/mistral/mistral-7b-instruct-v0.1.Q8_0.gguf


In [13]:
from llama_cpp import Llama
llm_mistral = Llama(model_path="models/mistral/mistral-7b-instruct-v0.1.Q8_0.gguf",
            n_gpu_layers=32, n_ctx=4096, verbose=True)

llama_model_load_from_file_impl: using device Metal (Apple M4) - 10472 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral/mistral-7b-instruct-v0.1.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.di

In [14]:
response = llm_mistral("### У вас получилось установить llama.cpp на GPU?", max_tokens=64)
print(response["choices"][0]["text"])

llama_perf_context_print:        load time =     403.81 ms
llama_perf_context_print: prompt eval time =     400.29 ms /    18 tokens (   22.24 ms per token,    44.97 tokens per second)
llama_perf_context_print:        eval time =    5104.47 ms /    63 runs   (   81.02 ms per token,    12.34 tokens per second)
llama_perf_context_print:       total time =    5522.89 ms /    81 tokens




Вопрос: У вас получилось установить llama.cpp на GPU?

Если да, то как и с чем?

Спасибо за ответ

Comment: Вам не удалось установить llama.cpp на GPU


### а логи говорят иное)